In [1]:
import pandas as pd
import calmap
import numpy as np
import os

In [7]:
from pathlib import Path
from os import environ, write

import re
from datetime import datetime, time
import datetime as dt
from collections import defaultdict

import requests

In [8]:
def load_data():
    if environ.get("USE_LOCAL_FILES"):
        paths = Path("extracted").rglob("*.csv")
        df_parts = []

        for path in sorted(paths):

            print("Reading for analysis:", path)

            with open(path, "r", encoding="utf-8") as fp:
                df_part = pd.read_csv(fp, delimiter=",", dtype=str)
                df_parts.append(df_part)

            # if df is None:
            #     df = df_part
            # else:
            #     df = df.append(df_part, ignore_index=True)

        df = pd.concat(df_parts, ignore_index=True)

    else:
        raise NotImplementedError("Remote data not yet implemented")
    
    df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], errors = 'coerce')
    df['date_of_proceeding'] = pd.to_datetime(df['date_of_proceeding'], errors = 'coerce')
    df['date_of_publication'] = pd.to_datetime(df['date_of_publication'], errors = 'coerce')

    return df

In [9]:
df = load_data()

Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-01-17T08-05-41.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-01-24T08-05-43.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-01-31T03-50-40.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-02-07T03-47-56.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-02-14T03-54-29.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-02-21T04-08-08.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-02-28T04-20-16.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-03-07T07-20-48.jsonl.csv
Reading for analysis: extracted/buckets/insolven

Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-20T00-26-46.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-20T07-31-07.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-20T13-43-00.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-20T20-17-09.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-21T06-38-42.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-21T13-46-12.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-21T19-37-11.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-22T07-02-46.jsonl.csv
Reading for analysis: extracted/buckets/insolven

In [10]:
df.drop_duplicates(subset ="description_hash", keep = "last", inplace = True)

In [103]:
df.to_csv('insolvenzen.csv')

In [92]:
df.tail()

,_key,case_nr,court,date_of_birth,date_of_proceeding,date_of_publication,description_hash,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint
1819827,NaN,810 IN 374/21 M-16-8,Frankfurt am Main,1982-09-30,2021-10-26,2021-10-30,48a4f96cdc782f1b2286b5ab2419375ce23d5c8c042656...,Hessen,NaN,in,NaN,Entscheidungen_im_Verfahren,60439,tbl_ergebnis:1:frm_detail,neu,"Malik, Samina",NaN,9966ad46f916f98b3f81231709710414b37224e481c2c5...
1819828,NaN,810 IK 23/21 C-16-7,Frankfurt am Main,1994-03-04,2021-10-27,2021-10-30,5188b3913814c5074cccdb2ecd9167a8df9c0f5baade51...,Hessen,NaN,ik,NaN,Entscheidungen_im_Verfahren,65931,tbl_ergebnis:0:frm_detail,neu,"Chebbour, Sid Ali Andre",NaN,a0ea1366c799bdbd43ee72b7999b2debf376775db1dee8...
1819829,NaN,10 IN 4/21,Wiesbaden,NaT,2021-10-28,2021-10-30,957706fffe11b03b85802de73b8d64e22c03c0b1b37a85...,Hessen,NaN,in,NaN,Abweisungen_mangels_Masse,65187,tbl_ergebnis:0:frm_detail,neu,"Meyer, Giuliano Rolf",NaN,92b5c14dea5c5e43fcca9c68ed28764c74de759bc2958c...
1819830,NaN,10 IK 137/20,Wiesbaden,1972-03-01,2021-10-28,2021-10-31,c518f8f5029dcfbce6ca57d53b1495e3add7cd60bfbd90...,Hessen,NaN,ik,NaN,Entscheidungen_im_Verfahren,65185,tbl_ergebnis:0:frm_detail,neu,"Hasanovic, Slavica",NaN,08687a7c1f1a39cad51dd7bcaf2cdb1b29ed63db31af71...
1819831,NaN,1 IK 1126/21,Mannheim,NaT,2021-10-31,2021-10-31,50f2aa4605f8d8a6ce42aa721585acc5606a20f6b32dc4...,Baden-Württemberg,NaN,ik,NaN,Verteilungsverzeichnisse_(§_188_InsO)_d._Verw....,NaN,tbl_ergebnis:0:frm_detail,neu,"Mayer, Tobias",NaN,2692683012ed351d9e15d5558fc9d438d594459ad641cb...


In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 840283 entries, 159 to 1819831
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   _key                 34818 non-null   object        
 1   case_nr              840283 non-null  object        
 2   court                840283 non-null  object        
 3   date_of_birth        601075 non-null  datetime64[ns]
 4   date_of_proceeding   744511 non-null  datetime64[ns]
 5   date_of_publication  840283 non-null  datetime64[ns]
 6   description_hash     840283 non-null  object        
 7   federal_state        840283 non-null  object        
 8   file_name            430671 non-null  object        
 9   kind                 839738 non-null  object        
 10  title                430671 non-null  object        
 11  type_of_proceeding   840283 non-null  object        
 12  zipcode              836266 non-null  object        
 13  detail_form

In [94]:
df.isnull().sum()

_key                   805465
case_nr                     0
court                       0
date_of_birth          239208
date_of_proceeding      95772
date_of_publication         0
description_hash            0
federal_state               0
file_name              409612
kind                      545
title                  409612
type_of_proceeding          0
zipcode                  4017
detail_form_name       430671
format                 430671
name                   430671
register               776585
request_fingerprint    430671
dtype: int64

## Filtern nach Verbraucherinsolvenz

In [6]:
# Funktion in_kind mit dem Argument/Wert df wird definiert

def ik_kind(df):
    ik_unternehmen = (df[df['kind'] == 'in'])
    return ik_unternehmen

# Funktion def wird ausgeführt
data_ik = ik_kind(df)

In [28]:
# Schleswig-Holstein, Hamburg, Niedersachsen, Bremen, Mecklenburg-Vorpommern

In [29]:
data_nw_ik = data_ik[data_ik["federal_state"].isin(['Nordrhein-Westfalen', "nw"])]

In [30]:
def ik_eröffnung(data_nw_ik):
    ik_eröffnet = data_nw_ik[data_ik['type_of_proceeding'].isin(['Eröffnungen', 'Eroeffnung', 'Eroeffnung_Insolvenzverfahren'])]
    return ik_eröffnet

# Funktion def wird ausgeführt
data_nw_eröffnet = ik_eröffnung(data_nw_ik)

<ipython-input-30-e7df5dfbe5a7>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ik_eröffnet = data_nw_ik[data_ik['type_of_proceeding'].isin(['Eröffnungen', 'Eroeffnung', 'Eroeffnung_Insolvenzverfahren'])]


In [11]:
publication_frequency_eröffent = data_nw_eröffnet.groupby('date_of_publication').date_of_publication.count().reset_index(name="count")

In [12]:
publication_frequency_eröffent.to_csv('freq_ik_eröffnet_neuneuneu.csv')

In [19]:
data_week_ik = (
    data_nw_eröffnet[["date_of_publication", "description_hash"]]
    .groupby(pd.Grouper(key="date_of_publication", freq="MS", label="left"))
    .count()
)

In [20]:
data_week_ik.to_csv('freq_in_eröffnet_bund_month.csv')

# SCHLUSS

In [13]:
data_week_ik.head()

,description_hash
date_of_publication,
2019-01-01,150
2019-02-01,122
2019-03-01,129
2019-04-01,167
2019-05-01,189


In [7]:
# Funktion in_kind mit dem Argument/Wert df wird definiert
def ik_kind(df):
    ik_privat = (df[df['kind'] == 'ik'])
    return ik_privat

# Funktion def wird ausgeführt
data_ik = ik_kind(df)

In [8]:
new_data_ik = data_ik.replace(['Nordrhein-Westfalen'], 'nw')

In [9]:
new_data_ik.federal_state.unique()

array(['ns', 'he', 'nw', 'hh', 'mv', 'rp', 'by', 'st', 'be', 'sl', 'th',
       'hb', 'sh', 'bb', 'sn', 'bw', 'Bayern', 'Rheinland-Pfalz',
       'Thüringen', 'Schleswig-Holstein', 'Sachsen-Anhalt', 'Sachsen',
       'Saarland', 'Niedersachsen', 'Mecklenburg-Vorpommern', 'Hessen',
       'Hamburg', 'Bremen', 'Brandenburg', 'Berlin', 'Baden-Württemberg'],
      dtype=object)

In [10]:
data_ik_nw = new_data_ik[(new_data_ik.federal_state == 'nw')]

In [24]:
data_ik_nw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192273 entries, 14 to 1099053
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   _key                 15070 non-null   object        
 1   case_nr              192273 non-null  object        
 2   court                192273 non-null  object        
 3   date_of_birth        105656 non-null  datetime64[ns]
 4   date_of_proceeding   191990 non-null  datetime64[ns]
 5   date_of_publication  192273 non-null  datetime64[ns]
 6   federal_state        192273 non-null  object        
 7   file_name            170666 non-null  object        
 8   kind                 192273 non-null  object        
 9   title                170666 non-null  object        
 10  type_of_proceeding   192273 non-null  object        
 11  zipcode              192273 non-null  object        
 12  detail_form_name     21607 non-null   object        
 13  format      

In [26]:
#freq_in_nw_eröffnung['zipcode'] = freq_in_nw_eröffnung['zipcode'].astype(str)
data_ik_nw.loc['zipcode'] = data_ik_nw['zipcode'].astype(str)

<ipython-input-26-e06f5d88c44c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ik_nw['zipcode'] = data_ik_nw['zipcode'].astype(str)


### Anzahl der Vorgänge bei Unternehmensinsolvenzen von Januar bis Mitte September

In [13]:
#data_nw_in.type_of_proceeding.unique()
data_ik_nw['type_of_proceeding'].value_counts()

Restschuldbefreiung                                      81777
Eroeffnung_Insolvenzverfahren                            22534
Entscheidungen_im_Verfahren                              22475
Entscheidungen_im_Verfahren_Aufhebung_Einstellung        15683
Entscheidungen_im_Verfahren_mit_Termine                  13271
Verwalter_Treuhaender_Verteilungsverzeichnis             12623
Bestimmung_Termine                                        6481
Entscheidungen_im_Restschuldbefreiungsverfahren           6138
Eröffnungen                                               4140
Restschuldbefreiung_Erteilung_Versagung                   3050
Verteilungsverzeichnisse_(§_188_InsO)_d._Verw./Treuh.     2318
Verteilungsverzeichnis_Nachtragsvert_Loeschung             517
Sonstiges                                                  510
Sonstiges_ausserhalb_des_Verfahresn                        474
Entscheidungen_nach_Aufhebung_des_Verfahrens               158
Abweisungen_mangels_Masse                              

### Privatinsolvenz Eröffnungen von 01.01. - 10.09. in NRW 

In [17]:
def ik_eröffnung(data_ik_nw):
    ik_eröffnet = (data_ik_nw[data_ik_nw['type_of_proceeding'] == 'Eroeffnung_Insolvenzverfahren'])
    return ik_eröffnet

# Funktion def wird ausgeführt
data_nw_eröffnet = ik_eröffnung(data_ik_nw)

In [22]:
data_nw_eröffnet.sample(5)

,_key,case_nr,court,date_of_birth,date_of_proceeding,date_of_publication,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint
191042,NaN,070f_IK00059_21,agkoeln,1962-10-27,2021-02-05,2021-02-08,nw,gerichte/nw/agkoeln/21/070f_IK00059_21/2021_02...,ik,"Kornett, Daniel, Wiehl, 70f IK 59/21",Eroeffnung_Insolvenzverfahren,51674,NaN,NaN,NaN,NaN,NaN
803275,NaN,0166_IK00093_21,agessen,1994-12-26,2021-05-11,2021-05-25,nw,gerichte/nw/agessen/21/0166_IK00093_21/2021_05...,ik,"Reh, Philipp, Essen, 166 IK 93/21",Eroeffnung_Insolvenzverfahren,45307,NaN,NaN,NaN,NaN,NaN
626973,NaN,0096_IK00064_21,agbonn,1991-09-13,2021-04-07,2021-04-13,nw,gerichte/nw/agbonn/21/0096_IK00064_21/2021_04_...,ik,"Fabri, Josef Michael, Much, 96 IK 64/21",Eroeffnung_Insolvenzverfahren,53804,NaN,NaN,NaN,NaN,NaN
688659,NaN,070d_IK00039_21,agkoeln,1956-05-01,2021-04-30,2021-05-04,nw,gerichte/nw/agkoeln/21/070d_IK00039_21/2021_05...,ik,"Hommes, Hannelore Annemarie, Köln, 70d IK 39/21",Eroeffnung_Insolvenzverfahren,51103,NaN,NaN,NaN,NaN,NaN
857461,NaN,070f_IK00139_21,agkoeln,1988-12-07,2021-05-21,2021-05-25,nw,gerichte/nw/agkoeln/21/070f_IK00139_21/2021_05...,ik,"Junker, Stefanie, Wesseling, 70f IK 139/21",Eroeffnung_Insolvenzverfahren,50389,NaN,NaN,NaN,NaN,NaN


#### Groupby date_of-Publication

In [27]:
freq_ik_nw_eröffnung = data_nw_eröffnet.groupby(['date_of_publication'], as_index=False).count() 
freq_ik_nw_eröffnung.sample(5)

,date_of_publication,_key,case_nr,court,date_of_birth,date_of_proceeding,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint
100,2021-05-20,0,186,186,182,186,186,186,186,186,186,186,0,0,0,0,0
45,2021-03-05,0,148,148,146,148,148,148,148,148,148,148,0,0,0,0,0
84,2021-04-28,0,164,164,162,164,164,164,164,164,164,164,0,0,0,0,0
9,2021-01-14,94,94,94,94,94,94,94,94,94,94,94,0,0,0,0,0
103,2021-05-25,0,136,136,136,136,136,136,136,136,136,136,0,0,0,0,0


In [29]:
freq_ik_nw_eröffnung.to_csv('freq_ik_nw_eröffnung_public.csv')

#### Groupby date_of_birth

In [31]:
freq_ik_nw_eröffnung_birth = data_nw_eröffnet.groupby(['date_of_birth'], as_index=False).count() 
freq_ik_nw_eröffnung_birth.sample(5)

,date_of_birth,_key,case_nr,court,date_of_proceeding,date_of_publication,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint
7509,1994-08-08,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0
6773,1990-12-06,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0
6168,1988-03-08,0,6,6,6,6,6,6,6,6,6,6,0,0,0,0,0
1935,1966-02-20,1,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0
739,1957-07-20,0,4,4,4,4,4,4,4,4,4,4,0,0,0,0,0


In [33]:
freq_ik_nw_eröffnung_birth.to_csv('freq_ik_nw_eröffnung_birth.csv')

#### Restschuld Privatunternehmen

In [38]:
def ik_restschuld(data_ik_nw):
    ik_rest = (data_ik_nw[data_ik_nw['type_of_proceeding'] == 'Restschuldbefreiung'])
    return ik_rest

# Funktion def wird ausgeführt
restschuld_ik = ik_restschuld(data_ik_nw)

In [39]:
restschuld_ik.describe()

,register
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


### Bundesweit

In [84]:
def ik_eröffnung(data_ik):
    ik_eröffnet = data_ik[data_ik['type_of_proceeding'].isin(['Eröffnungen', 'Eroeffnung', 'Eroeffnung_Insolvenzverfahren'])]
    return ik_eröffnet

# Funktion def wird ausgeführt
data_eröffnet = ik_eröffnung(data_ik)